In [1]:
import pandas as pd
import numpy as np
import os
import glob
sub1_path = 'D:/!BraVa_src/juntentou/sub01/'
sub2_path = 'D:/!BraVa_src/juntentou/sub02/'

print (glob.glob(sub1_path + '*.csv'))
print (glob.glob(sub2_path + '*.csv'))

sub1_0 = pd.read_csv(glob.glob(sub1_path + '*.csv')[0])
sub1_1 = pd.read_csv(glob.glob(sub1_path + '*.csv')[1])
sub2_0 = pd.read_csv(glob.glob(sub2_path + '*.csv')[0])
sub2_1 = pd.read_csv(glob.glob(sub2_path + '*.csv')[1])

# 定义函数：保留到第二个 r=0 行之前的数据
def filter_before_second_r_zero(df):
    r_zero_count = 0
    second_r_zero_index = None

    for index, row in df.iterrows():
        if row['r'] == 0:
            r_zero_count += 1
            if r_zero_count == 2:  # 找到第二个 r = 0 的行
                second_r_zero_index = index
                break

    # 如果找到第二个 r=0 行，则保留之前的数据
    if second_r_zero_index is not None:
        df = df.iloc[:second_r_zero_index]
    
    return df

# 对所有变量应用该函数
# sub1_0 = filter_before_second_r_zero(sub1_0)
# sub1_1 = filter_before_second_r_zero(sub1_1)
# sub2_0 = filter_before_second_r_zero(sub2_0)
# sub2_1 = filter_before_second_r_zero(sub2_1)


['D:/!BraVa_src/juntentou/sub01\\40+67+68_mirrored_g.csv', 'D:/!BraVa_src/juntentou/sub01\\47+60+61.csv']
['D:/!BraVa_src/juntentou/sub02\\40+68+66_mirrored_g.csv', 'D:/!BraVa_src/juntentou/sub02\\47+60+61.csv']


In [2]:
# 生成VTK文件内容
def generate_vtk(data, filename):
    with open(filename, 'w') as vtk_file:
        vtk_file.write("# vtk DataFile Version 3.0\n")
        vtk_file.write("Curve data\n")
        vtk_file.write("ASCII\n")
        vtk_file.write("DATASET POLYDATA\n")
        
        # 写入点数据
        num_points = len(data)
        vtk_file.write(f"POINTS {num_points} float\n")
        for index, row in data.iterrows():
            vtk_file.write(f"{row['x']} {row['y']} {row['z']}\n")
        
        # 写入拓扑数据
        vtk_file.write(f"LINES 1 {num_points + 1}\n")
        vtk_file.write(f"{num_points} " + " ".join(map(str, range(num_points))) + "\n")
        
        # 写入点数据属性
        vtk_file.write(f"POINT_DATA {num_points}\n")
        
        # 写入 r 数据
        vtk_file.write("SCALARS r float 1\n")
        vtk_file.write("LOOKUP_TABLE default\n")
        for value in data['r']:
            vtk_file.write(f"{value}\n")
        
        # 写入 curvature 数据
        vtk_file.write("SCALARS curvature float 1\n")
        vtk_file.write("LOOKUP_TABLE default\n")
        for value in data['curvature']:
            vtk_file.write(f"{value}\n")
        
        # 写入 torsion 数据
        vtk_file.write("SCALARS torsion float 1\n")
        vtk_file.write("LOOKUP_TABLE default\n")
        for value in data['torsion']:
            vtk_file.write(f"{value}\n")

# 调用函数生成 VTK 文件

generate_vtk(sub1_0, 'sub1_0.vtk')
generate_vtk(sub1_1, 'sub1_1.vtk')
generate_vtk(sub2_0, 'sub2_0.vtk')
generate_vtk(sub2_1, 'sub2_1.vtk')